In [12]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)


In [ ]:
grouped = df.groupby("smoking")
df_0 = grouped.get_group(0)
df_1 = grouped.get_group(1)
df_0_half = df_0.sample(frac=0.5, random_state=42)
df_1_half = df_1.sample(frac=0.5, random_state=42)
df_half = pd.concat([df_0_half, df_1_half])

df_half = df_half.sample(frac=1, random_state=42).reset_index(drop=True)


In [15]:
df_half['smoking'].value_counts()

smoking
0    33845
1    33845
Name: count, dtype: int64

In [16]:

train_copy=train.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test= test_set.drop('smoking', axis=1)
y_test = test_set['smoking']


In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 100, 300, step=20)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)

    scores = []
    F1=[]

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        #
        # scaler = MinMaxScaler()
        # X_train_scaled = scaler.fit_transform(X_train)
        # X_val_scaled = scaler.transform(X_val)

        
        model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_jobs=-1,
            random_state=42
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        acc = accuracy_score(y_val, y_pred)
        
        F1c=f1_score(y_val, y_pred, average='macro') 
        F1.append(F1c)
        

        y_pred_train = model.predict(X_train)
        acc_train = accuracy_score(y_train, y_pred_train)
        f1_train = f1_score(y_train, y_pred_train, average='macro')
        

        overfit_penalty = abs(acc_train - acc)
        score = 0.5 * acc - 1 * overfit_penalty
        
        
        
        scores.append(score)

    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize',sampler=sampler)
study.optimize(objective, n_trials=20)  


print("best_params:", study.best_params)
print(f"best_value: {study.best_value:.4f}")

[I 2025-07-29 21:31:27,448] A new study created in memory with name: no-name-a7f13d44-41e2-4e73-a720-82599d0b70c4
[I 2025-07-29 21:31:33,959] Trial 0 finished with value: 0.3482309055990545 and parameters: {'n_estimators': 160, 'max_depth': 10, 'learning_rate': 0.012475943657312591}. Best is trial 0 with value: 0.3482309055990545.
[I 2025-07-29 21:31:37,671] Trial 1 finished with value: 0.3645017727877088 and parameters: {'n_estimators': 120, 'max_depth': 9, 'learning_rate': 0.012084361179543377}. Best is trial 1 with value: 0.3645017727877088.
[I 2025-07-29 21:31:39,744] Trial 2 finished with value: 0.3533018171074015 and parameters: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.2743396987416097}. Best is trial 1 with value: 0.3645017727877088.
[I 2025-07-29 21:31:41,011] Trial 3 finished with value: 0.3806655340522972 and parameters: {'n_estimators': 140, 'max_depth': 3, 'learning_rate': 0.016809674773986334}. Best is trial 3 with value: 0.3806655340522972.
[I 2025-07-29 2

最佳参数组合: {'n_estimators': 240, 'max_depth': 4, 'learning_rate': 0.040958312489590074}
对应平均准确率: 0.3883


In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

   
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

  
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=10,
        learning_rate=0.1,
        n_estimators=100,
        eval_metric='logloss',
        random_state=42
    )

    model.fit(X_train_scaled, y_train)


    y_pred = model.predict(X_val_scaled)
    acc = accuracy_score(y_val, y_pred)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")
    scores.append(acc)


avg_acc = np.mean(scores)
print(f"\navg_acc: {avg_acc:.4f}")

In [ ]:
model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=4,
        learning_rate=0.040958312489590074,
        n_estimators=240,
        eval_metric='logloss',
        random_state=42
    )
model.fit(X, y)  

y_test_pred = model.predict(X_test)

acc_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='macro')
recall_test = recall_score(y_test, y_test_pred, average='macro')
f1_test = f1_score(y_test, y_test_pred, average='macro')

train_pred = model.predict(X)  # 用整个训练集来预测
train_acc = accuracy_score(y, train_pred)
print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

print(f"testAccuracy:  {acc_test:.4f}")
print(f"test Precision: {precision_test:.4f}")
print(f"test Recall:    {recall_test:.4f}")
print(f"test F1 Score:  {f1_test:.4f}")
print("\ntest report")
print(classification_report(y_test, y_test_pred, digits=2))

Train Accuracy: 0.7881, test Accuracy: 0.7945
测试集 Accuracy:  0.7945
测试集 Precision: 0.8036
测试集 Recall:    0.7945
测试集 F1 Score:  0.7930

测试集分类报告:
              precision    recall  f1-score   support

           0       0.86      0.71      0.78      1000
           1       0.75      0.88      0.81      1000

    accuracy                           0.79      2000
   macro avg       0.80      0.79      0.79      2000
weighted avg       0.80      0.79      0.79      2000

